# Create runtime and compute for the prompt flow

In [ ]:
# import required libraries
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment, BuildContext
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

from azure.ai.ml.entities import ComputeInstance 
from azure.ai.ml.entities import CustomApplications, ImageSettings, EndpointsSettings, VolumeSettings 



Create custom environment for runtime

In [ ]:
ml_client = MLClient.from_config(credential=credential)
env_docker_context = Environment(
    build=BuildContext(path="docker"),
    name="promptflow-contoso-chat",
    description="Custom promptflow contoso chat environment",
)
ml_client.environments.create_or_update(env_docker_context)

In [ ]:
envs = ml_client.environments.get(name="promptflow-contoso-chat", version="2")
print(envs)

Now we can create the compute and env we set up.

In [ ]:
#image= ImageSettings(reference="promptflow-contoso-chat:latest")
image= ImageSettings(reference="mcr.microsoft.com/azureml/promptflow/promptflow-runtime:20231114.v2")
endpoints = [EndpointsSettings(published=8081, target=8080)]
app = CustomApplications(name='promptflow-runtime-contoso-chat',endpoints=endpoints,bind_mounts=[],image=image,environment_variables={}) 

# create compute instance
ci_basic_name = "promptflow-contoso-chat"
ci_basic = ComputeInstance(name=ci_basic_name, size="STANDARD_DS12_V2",custom_applications=[app]) 
ml_client.begin_create_or_update(ci_basic)

Create the runtime

Once the compute instance and enviornment is created. [Follow these instructions to create a runtime in AI Studio.](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/create-manage-runtime)